В этом туториале будем готовить данные для последующего обучения word2vec

Возьмём тот же корпус, что и в предыдущем туториале

In [ ]:
import json
from string import punctuation
import random

from datasets import load_dataset
import nltk
from nltk.tokenize import ToktokTokenizer
import numpy as np
import pandas as pd
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm

morph = MorphAnalyzer()
tokenizer = ToktokTokenizer()
punct = punctuation + "«»—"

In [ ]:
corpus = load_dataset('cedr')

In [ ]:
corpus = corpus['train']['text']
print(f'Текстов: {len(corpus)}')

In [ ]:
def tokenize(text):
    return tokenizer.tokenize(text.lower())

In [ ]:
corpus = [tokenize(text) for text in corpus]
print(f'Слов: {sum([len(t) for t in corpus])}')

## Соберем словарь лемм

Лемматизировать весь корпус будет очень долго. Давайте лучше соберем словарь уникальных слов и будет лемматизировать только уникальные слова.

In [ ]:
def lemmatize(word):
    word_data = morph.parse(word)[0]
    return word_data.normal_form

In [ ]:
tok2lemma = {}

for text in tqdm(corpus):
    for tok in text:
        if tok not in tok2lemma:
            tok2lemma[tok] = lemmatize(tok)

In [ ]:
# ключ - уникальное слово в нашем корпусе, значение - его лемма
tok2lemma['уехали']

In [ ]:
len(tok2lemma)

In [ ]:
stopwords = nltk.corpus.stopwords.words('russian')

# Замена слов леммами
Так как теперь к каждому слову из нашего корпуса мы знаем лемму, то давайте каждое слово заменим на его лемму и уберем стоп-слова и пунктуацию.
Это работает гораздо(!) быстрее, чем если бы мы в корпусе для каждого слова каждый раз рассчитывали лемму (с помощью пайморфи), 
потому что теперь нам надо вызвать пайморфи 29147 раз вместо 128233.

In [ ]:
lemmas_corpus = [[tok2lemma[tok] for tok in text if tok not in stopwords and tok and tok not in punct]
                 for text in tqdm(corpus)]

# Соберем частотный словарь

In [ ]:
freq = {}

for text in tqdm(lemmas_corpus):
    for tok in text:
        if tok in freq:
            freq[tok] += 1
        else:
            freq[tok] = 1

In [ ]:
freq_df = pd.DataFrame(data={'word': list(freq.keys()), 'n_entries': list(freq.values())})

In [ ]:
freq_df.sort_values(by=['n_entries'], ascending=False, inplace=True)

In [ ]:
freq_df.head()

In [ ]:
# уникальных слов в словаре
freq_df.shape

In [ ]:
freq_df.tail()

In [ ]:
n_words = freq_df.n_entries.sum()

# Замена редких слов
В нашем корпусе осталось много слов, которые встречаются очень редко. Давайте мы редкие слова заменим на специальный токен UNK - unknown. Так мы разительно сократим размер нашего словаря с незначительной потерей информации.

In [ ]:
print('Доля слов, которые мы заменим на UNK:')

for threshold in np.arange(5, 36, 5):
    
    sub_df = freq_df[freq_df.n_entries < threshold]
    
    unk_freq = sub_df['n_entries'].sum() * 100 / n_words
    
    print('Порог отсечения - {}, доля UNK - {:.2f} %, останется слов в словаре - {}, удалили - {} слов'.format(
        threshold, unk_freq, freq_df.shape[0] - sub_df.shape[0], sub_df.shape[0]))

In [ ]:
# корпус у нас не большой, поэтому возьмем минимум
threshold = 5

In [ ]:
vocab = freq_df[freq_df.n_entries >= threshold]

In [ ]:
words = set(vocab.word)

In [ ]:
len(words)

In [ ]:
'Мы сократили наш словарь в {:.2f} раз с потерей 1.51 % всех слов'.format(freq_df.shape[0] / len(words))

In [ ]:
def replace_with_unk(word):
    return word if word in words else 'UNK'

In [ ]:
# заменим слово токеном UNK, если его нет в нашем новом словаре
processed_corpus = [[replace_with_unk(tok) for tok in text] for text in tqdm(lemmas_corpus)]

In [ ]:
def drop_duplicate_unks(tokens):
    
    output_tokens = []
    
    for tok in tokens:
        
        if tok == 'UNK' and output_tokens and output_tokens[-1] == 'UNK':
            continue
            
        output_tokens.append(tok)
            
    return output_tokens

In [ ]:
sample_text = 'думать далее милый барышня UNK UNK тинькоф звонить неделя'.split()

In [ ]:
drop_duplicate_unks(sample_text)

In [ ]:
# дедублируем подряд идущие унки (оставим только один)
processed_corpus = [drop_duplicate_unks(sample) for sample in tqdm(processed_corpus)]

In [ ]:
texts_with_unk = [text for text in processed_corpus if 'UNK' in text]
'Текстов с унками - {:.2f} %'.format(len(texts_with_unk) * 100 / len(processed_corpus))

In [ ]:
# посмотрим на тексты с унками
for text in random.sample(texts_with_unk, k=5):
    print(' '.join(text))

In [ ]:
random.shuffle(processed_corpus)

In [ ]:
!mkdir data

In [ ]:
with open('data/processed_corpus.json', 'w') as f:
    json.dump(processed_corpus, f, ensure_ascii=False)